## Create PKS Kubernetes Cluster

### Authenticate to your PKS API and review existing clusters

In [88]:
pks


The Pivotal Container Service (PKS) CLI is used to create, manage, and delete Kubernetes clusters. To deploy workloads to a Kubernetes cluster created using the PKS CLI, use the Kubernetes CLI, kubectl.

Version: 1.4.0-build.230

Usage:
  pks [command]

Available Commands:
  cluster                View the details of the cluster
  clusters               Show all clusters created with PKS
  create-cluster         Creates a kubernetes cluster, requires cluster name, an external host name, and plan
  create-network-profile Create a network profile
  create-sink            Creates a sink for sending all log data to syslog://
  delete-cluster         Deletes a kubernetes cluster, requires cluster name
  delete-network-profile Delete a network profile
  delete-sink            Deletes a sink from the given cluster
  get-credentials        Allows you to connect to a cluster and use kubectl
  get-kubeconfig         Allows you to get kubeconfig for your username
  help                   Help ab

In [89]:
APIURL=api.us.azure.bekind.io
APIUSER=scott
APIPASS=changeme
pks login -a $APIURL -u $APIUSER -p $APIPASS -k


Error: Credentials were rejected, please try again.


: 1

In [91]:
pks clusters


Name        Plan Name  UUID                                  Status     Action
test-demo   small      77e8836c-1849-46ed-999c-26be9672bc0e  succeeded  CREATE
test-azure  small      7c2f04bc-a66e-4f7b-a9a4-79574bebb470  succeeded  CREATE



*Example Output:*

In [92]:
pks cluster test-demo


Name:                     test-demo
Plan Name:                small
UUID:                     77e8836c-1849-46ed-999c-26be9672bc0e
Last Action:              CREATE
Last Action State:        succeeded
Last Action Description:  Instance provisioning completed
Kubernetes Master Host:   demo.us.azure.bekind.io
Kubernetes Master Port:   8443
Worker Nodes:             3
Kubernetes Master IP(s):  10.0.16.20
Network Profile Name:     



In [94]:
pks plans


Name   ID                                    Description
small  8A0E21A8-8072-4D80-B365-D1F502085560  Single Master, 3 workers, privileged



### Create the PKS cluster

In [95]:
pks create-cluster


Error: accepts 1 arg(s), received 0
Usage:
  pks create-cluster <cluster-name> [flags]

Examples:
  pks create-cluster my-cluster --external-hostname example.hostname --plan production

Flags:
  -e, --external-hostname string   Address from which to access Kubernetes API
  -h, --help                       help for create-cluster
      --json                       Return the PKS-API output as json
      --network-profile string     Optional, network profile name (NSX-T only)
      --non-interactive            Don't ask for user input
  -n, --num-nodes string           Number of worker nodes
  -p, --plan string                Preconfigured plans. Run pks plans for more details
      --wait                       Wait for the operation to finish



: 1

### Decision time... 

Decide now how you will access the master node(s) of the resulting K8s cluster before you continue.

In my case, I have a domain us.azure.bekind.io into which I will manually create a DNS entry for a load balancer whose backend points to a pool of master node(s).

At the very least, if you specify an IP address in the "external-hostname," you have to have already acquired that public IP address on your cloud provider.  This IP or DNS name should be publically accessible.

In [96]:
pks create-cluster test-azure --external-hostname test.us.azure.bekind.io --plan small


Error: The cluster named "test-azure" already exists. Please use a different name.


: 1

*Example Output:*

Monitor `pks clusters` until the cluster creation is successful

In [97]:
pks clusters


Name        Plan Name  UUID                                  Status     Action
test-demo   small      77e8836c-1849-46ed-999c-26be9672bc0e  succeeded  CREATE
test-azure  small      7c2f04bc-a66e-4f7b-a9a4-79574bebb470  succeeded  CREATE



Capture the credentials for your new K8s cluster

In [99]:
pks get-credentials test-azure


Fetching credentials for cluster test-azure.
Context set for cluster test-azure.

You can now switch between clusters by using:
$kubectl config use-context <cluster-name>


Examine your new cluster

In [100]:
kubectl config use-context test-azure

Switched to context "test-azure".


In [101]:
kubectl cluster-info

Kubernetes master is running at https://test.us.azure.bekind.io:8443
CoreDNS is running at https://test.us.azure.bekind.io:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [102]:
kubectl get namespaces

NAME              STATUS   AGE
concourse         Active   67d
concourse-main    Active   67d
default           Active   70d
harbor            Active   64d
kpack             Active   67d
kube-node-lease   Active   70d
kube-public       Active   70d
kube-system       Active   70d
nginx-ingress     Active   67d
pks-system        Active   70d
wap-admin         Active   67d


Notice above the following namespaces DO NOT exist:
* vmware-system-tmc
* postfacto

# Attach the cluster to Tanzu Mission Control

Login to TMC via https://console.cloud.vmware.com

Download the TMC CLI, make it executable, and move it into your system path

In [103]:
tmc

____  _ ____
 | |\/| |___

Usage:
  tmc [resource|action|helper] [flags]

Resources:
  account       Account management.
  cluster       A Kubernetes cluster.
  clustergroup  A group of Kubernetes clusters.
  organization  Organization level management.
  workspace     A group of managed Kubenetes namespaces.

Actions:
  login         Login to Tanzu Mission Control.
  update        Update the CLI.

Helpers:
  completion    Generate a bash completion script.
  configure     Configure the current context environment
  iam           Permission management for resources.
  options       General options for Tanzu Mission Control.
  system        System level configuration and resources.
  version       Print CLI version.

Flags:
  -h, --help   help for tmc

Use "tmc [command] --help" for more information about a command.

Tanzu Mission Control v1.0
Copyright (c) 2020 VMware
https://vmware.bintray.com/tmc/tmc-eula.txt


In [104]:
# tmc login

*Example Output:*

In [106]:
tmc clustergroup list

  NAME      LABELS                
  cg-01                           
  cg-02                           
  cg-03                           
  cg-04                           
  cg-05                           
  cg-06                           
  cg-07                           
  cg-08                           
  cg-09                           
  cg-10                           
  default                         
  keith                           
  non-prod  environment:non-prod  
  prod      environment:prod      


Notice above the cluster group "cg-jbrightwell" does not exist

In [107]:
tmc cluster list

  NAME                  CLUSTERGROUP  PROVIDER  STATE  HEALTH   LABELS                                    
  c-01                  cg-01         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-02                  cg-02         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-03                  cg-03         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-04                  cg-04         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-05                  cg-05         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-07                  cg-07         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-08                  cg-08         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-09                  cg-09         AWS_EC2   READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-10                  cg-10        

Notice above the cluster "jbrightwell-c04" does not exist

In [109]:
tmc clustergroup templates list

  NAME         DESCRIPTION                                            
  default      A default ClusterGroup object.                         
  default-iam  default template for the iam-policy resource           
  generated    generated full template for the ClusterGroup resource  


## Create a new cluster group if desired

In [110]:
tmc clustergroup create -n cg-jbrightwell -t default

ℹ using template "default"
✔ clustergroup "cg-jbrightwell" created successfully 


In [111]:
tmc clustergroup list

  NAME            LABELS                
  cg-01                                 
  cg-02                                 
  cg-03                                 
  cg-04                                 
  cg-05                                 
  cg-06                                 
  cg-07                                 
  cg-08                                 
  cg-09                                 
  cg-10                                 
  cg-jbrightwell                        
  default                               
  keith                                 
  non-prod        environment:non-prod  
  prod            environment:prod      


## Attach the PKS cluster into the new Cluster Group

In [112]:
tmc cluster attach -g cg-jbrightwell -n jbrightwell-c04

✔ cluster "jbrightwell-c03" created successfully 
ℹ Run `kubectl apply -f k8s-attach-manifest.yaml` to attach the cluster


In [113]:
kubectl apply -f k8s-attach-manifest.yaml

namespace/vmware-system-tmc created
configmap/stack-config created
secret/tmc-client-secret created
customresourcedefinition.apiextensions.k8s.io/agents.clusters.tmc.cloud.vmware.com created
customresourcedefinition.apiextensions.k8s.io/extensions.clusters.tmc.cloud.vmware.com created
serviceaccount/extension-manager created
clusterrole.rbac.authorization.k8s.io/extension-manager-role created
clusterrolebinding.rbac.authorization.k8s.io/extension-manager-rolebinding created
service/extension-manager-service created
deployment.apps/extension-manager created
serviceaccount/extension-updater-serviceaccount created
clusterrole.rbac.authorization.k8s.io/extension-updater-clusterrole created
clusterrolebinding.rbac.authorization.k8s.io/extension-updater-clusterrolebinding created
service/extension-updater created
deployment.apps/extension-updater created
serviceaccount/agent-updater created
clusterrole.rbac.authorization.k8s.io/agent-updater-role created
clusterrolebinding.rbac.authorization

In [115]:
kubectl get namespaces

NAME                STATUS   AGE
concourse           Active   67d
concourse-main      Active   67d
default             Active   70d
harbor              Active   64d
kpack               Active   67d
kube-node-lease     Active   70d
kube-public         Active   70d
kube-system         Active   70d
nginx-ingress       Active   67d
pks-system          Active   70d
vmware-system-tmc   Active   12s
wap-admin           Active   67d


Notice the "vmware-system-tmc" namespace has been created.

In [120]:
kubectl -n vmware-system-tmc get all

NAME                                         READY   STATUS      RESTARTS   AGE
pod/agent-updater-65b6c5b7b7-wctvh           1/1     Running     0          47s
pod/agentupdater-workload-1580255520-t9xdl   0/1     Completed   0          35s
pod/data-protection-5cd89cd965-khwgb         1/1     Running     0          21s
pod/extension-manager-7f9b95f67b-shrdg       1/1     Running     0          49s
pod/extension-updater-66bb47f457-jzvcw       1/1     Running     0          48s
pod/inspection-extension-ccfc779b4-s8spg     1/1     Running     0          32s
pod/intent-agent-64858578d-zpnxd             1/1     Running     0          26s
pod/policy-sync-extension-68cf556c85-zm68l   1/1     Running     0          29s
pod/policy-webhook-64866489bd-gdws8          1/1     Running     1          29s
pod/policy-webhook-64866489bd-rqllh          1/1     Running     1          29s
pod/sync-agent-6564996cc7-jg4z6              1/1     Running     0          24s

NAME                                TYP

In [121]:
tmc clustergroup list

  NAME            LABELS                
  cg-01                                 
  cg-02                                 
  cg-03                                 
  cg-04                                 
  cg-05                                 
  cg-06                                 
  cg-07                                 
  cg-08                                 
  cg-09                                 
  cg-10                                 
  cg-jbrightwell                        
  default                               
  keith                                 
  non-prod        environment:non-prod  
  prod            environment:prod      


In [122]:
tmc cluster list --group cg-jbrightwell

  NAME                  CLUSTERGROUP    PROVIDER       STATE  HEALTH   LABELS                                    
  c-01                  cg-01           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-02                  cg-02           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-03                  cg-03           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-04                  cg-04           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-05                  cg-05           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-07                  cg-07           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-08                  cg-08           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.com/creator:bjung        
  c-09                  cg-09           AWS_EC2        READY  HEALTHY  tmc.cloud.vmware.

*This appears to be a bug.  The above command should scope search to 'cg-jbrightwell'*

In [123]:
open https://vmww1.tmc.cloud.vmware.com/clusters/jbrightwell-c04/global/overview

## Retrospective

In [124]:
open -a Firefox http://postfacto.us.azure.bekind.io

In [125]:
sh -c "cd charts/postfacto ; ./install-postfacto.sh"

namespace/postfacto created
NAME: postfacto
LAST DEPLOYED: Tue Jan 28 17:55:09 2020
NAMESPACE: postfacto
STATUS: deployed
REVISION: 1
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace postfacto -l "app.kubernetes.io/name=postfacto,app.kubernetes.io/instance=postfacto" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl port-forward $POD_NAME 8080:80
ingress.extensions/postfacto-ingress created
secret/ingress-tls-us-azure-bekind-io created


In [127]:
open https://vmww1.tmc.cloud.vmware.com/clusters/jbrightwell-c03/global/overview

In [128]:
kubectl -n postfacto get all

NAME                             READY   STATUS    RESTARTS   AGE
pod/postfacto-6dfcbb48bd-z4tjh   1/1     Running   0          92s
pod/postfacto-postgresql-0       1/1     Running   0          92s
pod/postfacto-redis-master-0     1/1     Running   0          92s

NAME                                    TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
service/postfacto                       ClusterIP   10.100.200.5    <none>        80/TCP     93s
service/postfacto-postgresql            ClusterIP   10.100.200.95   <none>        5432/TCP   93s
service/postfacto-postgresql-headless   ClusterIP   None            <none>        5432/TCP   93s
service/postfacto-redis-headless        ClusterIP   None            <none>        6379/TCP   93s
service/postfacto-redis-master          ClusterIP   10.100.200.53   <none>        6379/TCP   93s

NAME                        READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/postfacto   1/1     1            1           93s

NAME                 

In [129]:
kubectl -n postfacto \
  exec $(kubectl -n postfacto get pods -l "app.kubernetes.io/name"=postfacto -o=jsonpath='{.items..metadata.name}') \
  create-admin-user jbrightwell@vmware.com bekind

In [130]:
open -a Firefox https://postfacto.us.azure.bekind.io/admin

In [131]:
open -a Firefox https://postfacto.us.azure.bekind.io/retros/tmc

Fin